<a href="https://colab.research.google.com/github/MapleWolfe/Milestone_2/blob/main/Model_pipeline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### we are providing three ways to load the zip file data: please note that only one should be used.
### run cells for only one of these three but not all threegoogle bucket

### google bucket pre sets for data
google_application_credentials = '/content/organic-reef-390716-609989a4c6da.json'
bucket_name = 'fire_train_eval_test_bucket' #you need to name your bucket
zip_file_name = 'next_day_wildfire.zip' #name of the zip file

### google drive upload
google_drive_location = '/content/drive/MyDrive/next_day_wildfire.zip'

### if the zip file is uploaded to the same location as this notebook, start from the unzip cell:
load_local_zip = '/content/next_day_wildfire.zip'

### in the situation that any of the above code does not work:
### please look under Unzipping file section of the notebook

# Model pipeline 1

Hardware/system requirements:
- CPU - minimum 64 cores
- GPU - 2 Nvidia T4
- RAM - Minimum 400 GB
- Disk - Minimum 500 GB

This notebook will take upto 24hrs depending on your hardware specifications

## installs, imports, pre-sets

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py
!pip install google-cloud-storage

In [31]:
#google import options
from google.colab import drive
from google.cloud import storage

#general usage imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import multiprocessing
import pickle
import json
import joblib
import zipfile

#image processing
import skimage
from scipy.ndimage import distance_transform_edt

#sklearn preprocessing operations imports
from sklearn.preprocessing import StandardScaler

# unsupervised learning models
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans

# model evaluation and metrics
from sklearn.metrics import precision_score,recall_score, f1_score, accuracy_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.model_selection import KFold

#deeplearning
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense

#sklearn classifiers
from sklearn.linear_model import SGDClassifier
# sklearn regression
from sklearn.linear_model import Lasso

#GPU imports
import xgboost as xgb
import cudf
import cupy
from cuml.naive_bayes import GaussianNB
from cuml.naive_bayes import ComplementNB
from cuml import LogisticRegression
from cuml.ensemble import RandomForestClassifier

#viaualization library:
import matplotlib.pyplot as plt

In [49]:
num_cpus = multiprocessing.cpu_count()
print("Number of available CPUs:", num_cpus)

Number of available CPUs: 32


## Load Data using Google drive storage
(this works on the free google colab not the paid colab instance from gcp market place)

In [ ]:
# let's mount the drive
drive.mount('/content/drive')

# let's look into the zip file stored in the google drive
load_local_zip = google_drive_location


## Load Data from Google cloud storage

In [5]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = google_application_credentials
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(zip_file_name)
blob.download_to_filename(zip_file_name)
load_local_zip = zip_file_name

## Unzipping file
(from here our code begins regardless of source choice)

In [6]:
wildfire_zip =  zipfile.ZipFile(load_local_zip, 'r')
tf_record_file_names = wildfire_zip.namelist()

print('number of TF records:', len(tf_record_file_names))
print('file names of tf records within the zip:')
print(tf_record_file_names)

number of TF records: 19
file names of tf records within the zip:
['next_day_wildfire_spread_eval_00.tfrecord', 'next_day_wildfire_spread_eval_01.tfrecord', 'next_day_wildfire_spread_test_00.tfrecord', 'next_day_wildfire_spread_test_01.tfrecord', 'next_day_wildfire_spread_train_00.tfrecord', 'next_day_wildfire_spread_train_01.tfrecord', 'next_day_wildfire_spread_train_02.tfrecord', 'next_day_wildfire_spread_train_03.tfrecord', 'next_day_wildfire_spread_train_04.tfrecord', 'next_day_wildfire_spread_train_05.tfrecord', 'next_day_wildfire_spread_train_06.tfrecord', 'next_day_wildfire_spread_train_07.tfrecord', 'next_day_wildfire_spread_train_08.tfrecord', 'next_day_wildfire_spread_train_09.tfrecord', 'next_day_wildfire_spread_train_10.tfrecord', 'next_day_wildfire_spread_train_11.tfrecord', 'next_day_wildfire_spread_train_12.tfrecord', 'next_day_wildfire_spread_train_13.tfrecord', 'next_day_wildfire_spread_train_14.tfrecord']


### Functions to read into one tf record at a time by unziiping one file at a time

In [7]:
# unzipping one file at a time
def one_file_unzip(tf_record_file_name, zipfile_variable):
  extracted_record_path = zipfile_variable.extract(tf_record_file_name)
  raw_dataset = tf.data.TFRecordDataset(extracted_record_path)
  return raw_dataset

# yielding out one record at a time
def extract_one_row(tf_record_dataset):
  for i, raw_record in enumerate(tf_record_dataset.take(tf_record_dataset.cardinality().numpy())):
    one_record_dict = {}
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

    for key, feature in example.features.feature.items():

      kind = feature.WhichOneof('kind')
      one_record_dict[key] = np.array(getattr(feature, kind).value).reshape(64,64)
    yield one_record_dict

## Feature Generation

### feature description given by dataset provider

In [8]:
# data variables

INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph',
                  'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']

OUTPUT_FEATURES = ['FireMask']


# underlying feature value ranges:
# (min_clip, max_clip, mean, standard deviation)

feature_description_dict = {
    # Elevation in m: between 0.1 percentile and 99.9 percentile
    'elevation': (0.0, 3141.0, 657.3003, 649.0147),

    # Palmer Drought Severity Index: between 0.1 percentile and 99.9 percentile
    'pdsi': (-6.12974870967865, 7.876040384292651, -0.0052714925, 2.6823447),

    #Vegetation index times 10,000: between -1 and 1
    'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677),

    # Precipitation in mm: between 0.0 and 99.9 percentile
    'pr': (0.0, 44.53038024902344, 1.7398051, 4.482833),

    # Specific humidity: between 0 and 1
    'sph': (0., 1., 0.0071658953, 0.0042835088),

    # Wind direction in degrees clockwise from north: between 0 and 360.
    'th': (0., 360.0, 190.32976, 72.59854),

    #Min temp: between 253.15 kelvin and 99.9 percentile
    'tmmn': (253.15, 298.94891357421875, 281.08768, 8.982386),

    #Max temp: between 253.15 kelvin and 99.9 percentile
    'tmmx': (253.15, 315.09228515625, 295.17383, 9.815496),

    # Wind speed in m/s: between 0. and 99.9 percentile
    'vs': (0.0, 10.024310074806237, 3.8500874, 1.4109988),

    # NFDRS fire danger index energy release component BTU's per square foot.
    # 0., 99.9 percentile
    'erc': (0.0, 106.24891662597656, 37.326267, 20.846027),

    # Population density: between 0 and 99.9 percentile
    'population': (0., 2534.06298828125, 25.531384, 154.72331),

    # We don't want to normalize the FireMasks.
    # 1 indicates fire, 0 no fire, -1 unlabeled data
    'PrevFireMask': (-1., 1., 0., 1.),
    'FireMask': (-1., 1., 0., 1.)
}


### Feature Creation Functions

In [9]:
# lets define the min max scaling function
def min_max_scaling(array,min_val,max_val):
    scaled_array = np.clip((array - min_val) / (max_val - min_val), 0, 1)
    return scaled_array

# let's apply guassian smoothing
def gaussian_smoothing(image_array,sigma_val):
  smooth_array = skimage.filters.gaussian(image_array, sigma=1)
  return smooth_array

#lets get the rate of change and mean,
def local_pixel_features(image_array,radius_val):
  footprint = skimage.morphology.disk(radius_val)
  gradient_array = skimage.filters.rank.gradient(image_array, footprint)
  mean_array = skimage.filters.rank.mean(image_array, footprint)
  return gradient_array,mean_array

#use altitude edge to identify whether pixel is at a similar altitude as any pixel that has fire
def fire_pixel_shared_altitude(row_dict, normalized_array, previous_day_fire = 'PrevFireMask'):
  edges_array = skimage.feature.canny(normalized_array)
  inverted_edges_array = np.logical_not(edges_array).astype(int)
  edge_label_array = skimage.measure.label(inverted_edges_array)

  previous_fire = row_dict[previous_day_fire]
  fire_edge_labels = (edge_label_array*previous_fire)

  unique_regions_with_fire = np.unique(fire_edge_labels.flatten())
  non_zero_unique_regions = unique_regions_with_fire[unique_regions_with_fire != 0]

  fire_at_same_altitude = np.isin(edge_label_array, non_zero_unique_regions).astype(int)
  return fire_at_same_altitude

def distance_to_fire(row_dict,feature):
  # we need to clip the fire mask to account for -1 values (missing values where the satellite was unable to get a clear image)
  # for now we take them as no fire objects, however we will not be accounting for these pixels in our model.
  fire_mask_array = row_dict[feature].clip(0,1)
  inverted_mask_array = 1 - fire_mask_array
  distance_transform_array = distance_transform_edt(inverted_mask_array)
  return distance_transform_array


### Function to combine all features

In [10]:
# let's apply it on all features
def build_features(record_dict,min_max_dict,sigma_val,radius_val):
  feature_list = record_dict.keys()
  output_feature_dict = {}
  for a_feature in feature_list:
    if a_feature not in ['PrevFireMask','FireMask']:
     #min max scaling
     feature_min = min_max_dict[a_feature][0]
     feature_max = min_max_dict[a_feature][1]
     scaled_array = min_max_scaling(record_dict[a_feature],feature_min,feature_max)
     #guassian smoothing
     smoothen_array = gaussian_smoothing(scaled_array,sigma_val)

     #local pixel values: gradient values(rate of change), local mean val.
     gradient_array,mean_array = local_pixel_features(smoothen_array,radius_val)

     #lets now add these features to our output:
     output_feature_dict[a_feature+'_'+'scaled_smoothened_values'] = smoothen_array.flatten()
     output_feature_dict[a_feature+'_'+'local_gradient'] = gradient_array.flatten()
     output_feature_dict[a_feature+'_'+'local_mean'] = mean_array.flatten()

     #lets label pixels if they are at the same elevation (to account for cliffs/mountains/chasms) as the fire
     # here we aren't using smoothened array
    if a_feature == 'elevation':
      fire_at_altitude_array = fire_pixel_shared_altitude(record_dict, scaled_array)
      output_feature_dict['fire_at_similar_altitude'] = fire_at_altitude_array.flatten()
     #lets move are features into a dict.

    # get pixel eucledian distance from fire
    if a_feature == 'PrevFireMask':
      distance_array = distance_to_fire(record_dict,a_feature)
      output_feature_dict['PrevFireMask'] = record_dict[a_feature].flatten()
      output_feature_dict['distance_from_fire'] = distance_array.flatten()

    if a_feature == 'FireMask':
      output_feature_dict['FireMask'] = record_dict[a_feature].flatten()

  return output_feature_dict


### function that uses all prior functions and creates csv

In [11]:
def make_df_save_csv(string_file_name,tf_record_names = tf_record_file_names, main_zip_file = wildfire_zip, feature_descriptions= feature_description_dict):
  first_write = True
  total_rows = 0
  image_id = 0
  large_df_list = []
  column_names = []
  csv_file_name = string_file_name + '.csv'
  for a_tf_record in tf_record_names:
    if string_file_name in a_tf_record:
      print('started tf record: ', a_tf_record)
      raw_dataset = one_file_unzip(a_tf_record, main_zip_file)
      row_extraction_generator = extract_one_row(raw_dataset)
      single_record_list = []
      image_count = 0

      for a_row in row_extraction_generator:
        all_features_dict_array = build_features(a_row,feature_descriptions,sigma_val=1,radius_val=3)
        column_names = all_features_dict_array.keys()
        image_id += 1
        image_count +=1
        image_number_array = np.full(4096, image_id)
        all_features_dict_array['image_id'] = image_number_array

        if image_count == 1:
          all_features_dataframe = cudf.DataFrame.from_dict(all_features_dict_array)
        else:
          single_row_df = cudf.DataFrame.from_dict(all_features_dict_array)
          all_features_dataframe = all_features_dataframe.append(single_row_df, ignore_index=True)

        if image_count >= 200:
          single_record_list.append(all_features_dataframe)
          image_count = 0

      if image_count % 200 != 0:
        single_record_list.append(all_features_dataframe)
        image_count = 0

      big_df = cudf.concat(single_record_list, ignore_index=True)
      pandas_big_df = big_df.to_pandas()
      total_rows += len(pandas_big_df)
      if first_write == True:
        pandas_big_df.to_csv(csv_file_name, mode='w', index=False, header=True)
        first_write = False
      else:
        pandas_big_df.to_csv(csv_file_name, mode='a', index=False, header=False)

  print('completed: ', a_tf_record)
  print('csv output is complete')
  print('output csv lenght',total_rows)
  print('output csv image count: ', total_rows/4096)
  print('number of expected images', image_id)
  return None

### Creating test, train and eval csv files

In [ ]:
%%time
print('we are making eval csv')
make_df_save_csv('eval')

print('we are making test csv')
make_df_save_csv('test')

print('we are making train csv')
make_df_save_csv('train')

del wildfire_zip
gc.collect()

### Function to read CSV files

In [22]:
#remember to add .csv at the end of file name
def read_csv_in_chunks(file_name,number_images):
  pixels_count = 64*64
  size = number_images*pixels_count
  file_string = '/content/' + file_name
  return pd.read_csv(file_string, chunksize=size)

def read_full_csv(file_name):
  file_string = '/content/' + file_name
  return pd.read_csv(file_string)

### Function to create feature and target variables


In [14]:
def cleaner_1(df_chunk):
  col_list = ['NDVI_scaled_smoothened_values', 'NDVI_local_gradient', 'NDVI_local_mean', 'tmmn_scaled_smoothened_values', 'tmmn_local_gradient', 'tmmn_local_mean', 'elevation_scaled_smoothened_values', 'elevation_local_gradient', 'elevation_local_mean', 'fire_at_similar_altitude', 'population_scaled_smoothened_values', 'population_local_gradient', 'population_local_mean', 'vs_scaled_smoothened_values', 'vs_local_gradient', 'vs_local_mean', 'pdsi_scaled_smoothened_values', 'pdsi_local_gradient', 'pdsi_local_mean', 'pr_scaled_smoothened_values', 'pr_local_gradient', 'pr_local_mean', 'tmmx_scaled_smoothened_values', 'tmmx_local_gradient', 'tmmx_local_mean', 'sph_scaled_smoothened_values', 'sph_local_gradient', 'sph_local_mean', 'th_scaled_smoothened_values', 'th_local_gradient', 'th_local_mean', 'distance_from_fire', 'erc_scaled_smoothened_values', 'erc_local_gradient', 'erc_local_mean']

  original_previous_day_fire = df_chunk['PrevFireMask']
  original_next_day_fire = df_chunk['FireMask']

  #general cleaning for classifier and regressor
  drop_neg_df = df_chunk[df_chunk['FireMask'] >=0]

  #only regressor selection
  regressor_target = drop_neg_df['FireMask']

  #cleaning specifically for the classifier
  classifier_target = np.where(regressor_target > 0, 1, 0)
  dropped_chunk = drop_neg_df.drop(labels=['PrevFireMask','FireMask','image_id'], axis=1)

  return dropped_chunk[col_list],regressor_target,classifier_target, original_previous_day_fire, original_next_day_fire

### Applying the prior functions on the downloaded CSV files + scaling data

In [15]:
#train
train_df = read_full_csv('train.csv')
print('train loaded')
train_cleaned_df,train_regressor_target,train_classifier_target, train_original_previous_day_fire, train_original_next_day_fire = cleaner_1(train_df)
del train_df

gc.collect()

#evaluation
eval_df = read_full_csv('eval.csv')
print('eval loaded')
eval_cleaned_df,eval_regressor_target,eval_classifier_target, eval_original_previous_day_fire, eval_original_next_day_fire = cleaner_1(eval_df)
del eval_df
gc.collect()

#test
test_df = read_full_csv('test.csv')
print('test loaded')
test_cleaned_df,test_regressor_target,test_classifier_target, test_original_previous_day_fire, test_original_next_day_fire = cleaner_1(test_df)
del test_df
gc.collect()

train loaded
eval loaded
test loaded


15

## Unsupervised learning

### Standard Scaling

In [21]:
scaler = StandardScaler()

print('train data scaling started')
train_data_scaled = scaler.fit_transform(train_cleaned_df)
del train_cleaned_df
gc.collect()

print('eval data scaling started')
eval_data_scaled = scaler.transform(eval_cleaned_df)

print('test data scaling started')
test_data_scaled = scaler.transform(test_cleaned_df)
del test_cleaned_df
gc.collect()

train data scaling started
eval data scaling started
test data scaling started


6

### PCA

#### Building tuned PCA models

In [26]:
pca_param_grid = {'n_components': [4,6,8,10,12,14,16,18]}
pca_storage_dict = {}
model_counter = 0

for params in ParameterGrid(pca_param_grid):
  model_counter +=1
  print('initializing PCA for param: ', params)
  pca_model = PCA(**params)
  chunk_counter = 0
  pca_model.fit(train_data_scaled)
  print('pca completed for model : ', model_counter)

  pca_model_name_string = 'pca_model_'+str(model_counter)
  pca_storage_dict[pca_model_name_string] = [params,{'explained_variance': list(pca_model.explained_variance_)},{'explained_variance_ratio':list(pca_model.explained_variance_ratio_)}]
  print('storing pca file')
  with open(pca_model_name_string, 'wb') as pca_file:
    pickle.dump(pca_model, pca_file)

print('storing scalar model')
with open('standard_scalar_model', 'wb') as scaler_file:
  pickle.dump(scaler, scaler_file)

print('storing pca performance')
with open('pca_model_performance.json', 'w') as pca_metric_json:
    json.dump(pca_storage_dict, pca_metric_json)

initializing PCA for param:  {'n_components': 4}
pca completed for model :  1
storing pca file
initializing PCA for param:  {'n_components': 6}
pca completed for model :  2
storing pca file
initializing PCA for param:  {'n_components': 8}
pca completed for model :  3
storing pca file
initializing PCA for param:  {'n_components': 10}
pca completed for model :  4
storing pca file
initializing PCA for param:  {'n_components': 12}
pca completed for model :  5
storing pca file
initializing PCA for param:  {'n_components': 14}
pca completed for model :  6
storing pca file
initializing PCA for param:  {'n_components': 16}
pca completed for model :  7
storing pca file
initializing PCA for param:  {'n_components': 18}
pca completed for model :  8
storing pca file
storing scalar model
storing pca performance


#### PCA model evaluation plot

In [ ]:
pca_file = open('/content/pca_model_performance.json')
pca_models = json.load(pca_file)

x = list()
y = list()
for n,model in enumerate(pca_models.keys()):
    x.append(pca_models[model][0]['n_components'])
    idx = pca_models[model][0]['n_components'] - 1
    #y = np.sum(pca_models[model][1]['explained_variance'])
    y.append(np.cumsum(pca_models[model][2]['explained_variance_ratio'])[idx])

plt.plot(x,y,marker='o', linestyle='--', color='b')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

In [33]:
# the best PCA is Model 8 that. gives us about 18 components
with open('/content/pca_model_8', 'rb') as pca_file:
    loaded_pca_model = pickle.load(pca_file)

### Kmeans

#### K means cluster evaluation functions

In [34]:
def cluster_evaluation(eval_df, cluster_model):
    print('evaluation start')
    eval_labels = cluster_model.predict(eval_df)
    inertia = cluster_model.inertia_
    calinski = calinski_harabasz_score(eval_df, eval_labels)
    davies_bouldin = davies_bouldin_score(eval_df, eval_labels)
    # we are no longer calculating silhouette score as it performs pairwise calculations that grow exponentially with data
    #silhouette = silhouette_score(eval_df, eval_labels)
    print('evaluation complete')
    return inertia, calinski, davies_bouldin

#### K-means without PCA

In [56]:
def search_params_kmeans(file,scaling_model,pca_model,cluster_list=[8,32,64],initialisation_list = ['k-means++', 'random'], random_state = [0]):
  k_means_param_grid = {'n_clusters': cluster_list, 'init': initialisation_list, 'random_state' : random_state, 'batch_size' : [1024*num_cpus]}
  for params in ParameterGrid(k_means_param_grid):
    print('initializing kmeans for param: ', params)
    csv_chunks_generator = read_csv_in_chunks(file,1000)
    K_means_model = MiniBatchKMeans(**params)
    counter = 0
    for a_chunk in csv_chunks_generator:
      features_df,_,_,_,_ = cleaner_1(a_chunk)
      scaled_df = scaling_model.transform(features_df)
      K_means_model.partial_fit(scaled_df)
      print('iteration completed: ', counter)
      counter+=1
    yield K_means_model, params

In [ ]:
model_builders = search_params_kmeans(file='train.csv',scaling_model = scaler, pca_model = loaded_pca_model)
model_perform_dict ={}
model_counter = 0

#this where a lot of time will go, it will iterate over each model across grid search
for a_kmean_model, kmean_params in model_builders:
  model_counter +=1
  print('initializing evaluation')
  inertia, calinski, davies_bouldin = cluster_evaluation(eval_cleaned_df, a_kmean_model)
  print('evaluation complete')

  model_name = 'kmean_model_no_pca'+str(model_counter)
  model_perform_dict[model_name]=[kmean_params,inertia, calinski, davies_bouldin]
  print('storing model')
  with open(model_name, 'wb') as model_file:
    pickle.dump(a_kmean_model, model_file)

#outputing our evaluation metrics for all the models
with open('kmean_model_performance.json', 'w') as kmeans_metric_json:
    json.dump(model_perform_dict, kmeans_metric_json)


#### K-means with PCA

In [ ]:
def search_params_kmeans_pca(file,scaling_model,pca_model,cluster_list=[8,32,64],initialisation_list = ['k-means++', 'random'], random_state = [0]):
  k_means_param_grid = {'n_clusters': cluster_list, 'init': initialisation_list, 'random_state' : random_state, 'batch_size' : [1024*num_cpus]}
  for params in ParameterGrid(k_means_param_grid):
    print('initializing kmeans for param: ', params)
    csv_chunks_generator = read_csv_in_chunks(file,1000)
    K_means_model = MiniBatchKMeans(**params)
    counter = 0
    for a_chunk in csv_chunks_generator:
      features_df,_,_,_,_ = cleaner_1(a_chunk)
      scaled_df = scaling_model.transform(features_df)
      out_pca_df = pca_model.transform(scaled_df)
      K_means_model.partial_fit(out_pca_df)
      print('iteration completed: ', counter)
      counter+=1
    yield K_means_model, params

In [ ]:
model_builders = search_params_kmeans_pca(file='train.csv',scaling_model = scaler, pca_model = loaded_pca_model)
model_perform_dict_pca ={}
model_counter = 0
eval_pca = loaded_pca_model.transform(eval_data_scaled)
#this where a lot of time will go, it will iterate over each model across grid search
for a_kmean_model, kmean_params in model_builders:
  model_counter +=1
  print('initializing evaluation')
  inertia, calinski, davies_bouldin = cluster_evaluation(eval_pca, a_kmean_model)
  print('evaluation complete')

  model_name = 'kmean_model_'+str(model_counter)
  model_perform_dict_pca[model_name]=[kmean_params,inertia, calinski, davies_bouldin]
  print('storing model')
  with open(model_name, 'wb') as model_file:
    pickle.dump(a_kmean_model, model_file)
#outputing our evaluation metrics for all the models
with open('kmean_pca_model_performance.json', 'w') as kmeans_metric_json:
    json.dump(model_perform_dict, kmeans_metric_json)

#### k-means plot

In [ ]:
def read_json(path):
    kmeans_with_pca = path
    df_with_pca = pd.read_json(kmeans_with_pca)
    pca = df_with_pca.iloc[1:, ]
    result = pca.apply(np.vectorize(lambda x: x[1]))
    result['metrics']= ['inertia', 'Calinski', 'davies_bouldin']
    pca_df = result.set_index('metrics').transpose()
    normalized_df_wpca=(pca_df-pca_df.min())/(pca_df.max()-pca_df.min())
    return normalized_df_wpca

#create dataframe for plot
def metric_df(metric):
    metric_df = pd.DataFrame([df_wpca[metric], df_nopca[metric]]).transpose()
    metric_df.set_axis(['metric' + '_pca','metric' + '_no_pca'], axis = 1)
    return metric_df

#plot generation
def generate_bar(metric):
    ax = metric_df(metric).plot.bar()
    ax.set_ylabel('Normalized'+ metric +'Score')
    ax.legend(['PCA', 'no_PCA'])
    ax.set_title(metric + ' Comparison for models with/wo PCA')

    for tick in ax.get_xticklabels():
        tick.set_rotation(45)

In [ ]:
#model_performance_with_pca
df_wpca= read_json('/content/kmean_pca_model_performance.json')

#model_performance_no_pca
df_nopca= read_json('/content/kmean_model_performance.json')


In [ ]:
generate_bar('inertia')

In [ ]:
generate_bar('Calinski')

In [ ]:
generate_bar('davies_bouldin')

In [ ]:
#kmeans model chosen based on explanation in the report:
with open('/content/kmean_model_1', 'rb') as kmean_file:
    loaded_kmean_model = pickle.load(kmean_file)

## Supervised Learning

In [ ]:
with open('/content/standard_scalar_model', 'rb') as ss_file:
    loaded_scalar_model = pickle.load(ss_file)

### Data preperation for supervised training

In [ ]:
print('initializing train pca')
train_data_pca = loaded_pca_model.transform(train_data_scaled)
del train_data_scaled
gc.collect()

print('getting train cluster labels')
train_cluster_labels = loaded_kmean_model.predict(train_data_pca)

print('initializing eval pca')
eval_data_pca = loaded_pca_model.transform(eval_data_scaled)
del eval_data_scaled
gc.collect()

print('getting eval cluster labels')
eval_cluster_labels = loaded_kmean_model.predict(eval_data_pca)


print('initializing test pca')
test_data_pca = loaded_pca_model.transform(test_data_scaled)
del test_data_scaled
gc.collect()

print('getting test cluster labels')
test_cluster_labels = loaded_kmean_model.predict(test_data_pca)


### Breaking training data into clusters function

In [ ]:
def break_cluster(pca_data,cluster_labels,supervised_target):
  unique_labels = list(np.unique(cluster_labels))
  storage_dict={}
  for a_label in unique_labels:
    label_index = np.where(cluster_labels == a_label)[0]
    feature_data = pca_data[label_index]
    target_data = supervised_target[label_index]
    yield a_label,feature_data,target_data

### Classifier Model building

#### Logistic regression

In [62]:
def logistic_train(a_label,feature_df,target_array,penalty = ['l2']):
  model_counter = 0
  hyper_params ={'penalty': penalty}
  name_param = {}
  for params in ParameterGrid(hyper_params):
    print('cluster: ', a_label, ', logistic regression for params: ', params)
    model_counter+=1
    print('initializing training of logistic regression')
    lrs = LogisticRegression(**params)
    lrs.fit(feature_df,target_array)
    print('training complete')
    print('saving model...')
    model_name = 'cluster_'+str(a_label)+'_logistic_model_'+str(model_counter)
    with open(model_name, 'wb') as model_file:
      pickle.dump(lrs, model_file)
    print('model saved')
    name_param[model_name] = params
  return name_param


#### SGD CLASSIFIER

In [63]:
def sgd_train(a_label,feature_df,target_array,penalty = ['l1', 'l2', 'elasticnet'],random_state=[0],n_jobs=[-1]):
  model_counter = 0
  hyper_params ={'penalty': penalty,'random_state':random_state,'n_jobs':n_jobs}
  name_param = {}
  for params in ParameterGrid(hyper_params):
    print('cluster: ', a_label, ', SGD for params: ', params)
    model_counter+=1
    print('initializing training of SGD ')
    sgd = SGDClassifier(**params)
    sgd.fit(feature_df,target_array)
    print('training complete')
    print('saving model...')
    model_name = 'cluster_'+str(a_label)+'_SGD_model_'+str(model_counter)
    with open(model_name, 'wb') as model_file:
      pickle.dump(sgd, model_file)
    print('model saved')
    name_param[model_name] = params
  return name_param


#### Random Forest

In [64]:
def random_forest_train(a_label,feature_df,target_array,n_estimators = [100,200,400],min_samples_split = [4096],random_state=[0]):
  model_counter = 0
  hyper_params ={'n_estimators': n_estimators,'random_state':random_state}
  name_param = {}
  for params in ParameterGrid(hyper_params):
    print('cluster: ', a_label, ', random forest for params: ', params)
    model_counter+=1
    print('initializing training of random forest')
    rfc = RandomForestClassifier(**params)
    rfc.fit(feature_df,target_array)
    print('training complete')
    print('saving model...')
    model_name = 'cluster_'+str(a_label)+'_random_forest_model_'+str(model_counter)
    with open(model_name, 'wb') as model_file:
      pickle.dump(rfc, model_file)
    print('model saved')
    name_param[model_name] = params
  return name_param

#### XGB classifier

In [65]:
def xgb_train(a_label,feature_df,target_array,n_estimators = [100,200,400],tree_method =['gpu_hist'], objective = ['binary:logistic'] ):
  model_counter = 0
  hyper_params ={'n_estimators': n_estimators,'tree_method':tree_method, 'objective':objective}
  name_param = {}
  for params in ParameterGrid(hyper_params):
    print('cluster: ', a_label, ', xgb for params: ', params)
    model_counter+=1
    print('initializing training of random forest')
    xgc = xgb.XGBClassifier(**params)
    # optimizing for gpu usage
    xgc.fit(feature_df,target_array)
    print('training complete')
    print('saving model...')
    model_name = 'cluster_'+str(a_label)+'_xgb_model_'+str(model_counter)
    with open(model_name, 'wb') as model_file:
      pickle.dump(xgc, model_file)
    print('model saved')
    name_param[model_name] = params
  return name_param

#### Complement Naive Bayes

In [66]:
def complement_nb_train(a_label,feature_df,target_array):
  name_param = {}
  print('cluster: ', a_label, ', Complement for params: ',)
  model_counter=1
  print('initializing training of ComplementNB')
  cnb = GaussianNB()
  cnb.fit(feature_df,target_array)
  print('training complete')
  print('saving model...')
  model_name = 'cluster_'+str(a_label)+'_ComplementNB_'+str(model_counter)
  with open(model_name, 'wb') as model_file:
    pickle.dump(cnb, model_file)
  print('model saved')
  name_param[model_name] = 'default params'
  return name_param

#### Gaussian Naive Bayes

In [67]:
def gaussian_nb_train(a_label,feature_df,target_array):
  name_param = {}
  print('cluster: ', a_label, ', gaussian for params: ',)
  model_counter=1
  print('initializing training of guassianNB')
  gnb = GaussianNB()
  gnb.fit(feature_df,target_array)
  print('training complete')
  print('saving model...')
  model_name = 'cluster_'+str(a_label)+'_guassianNB_'+str(model_counter)
  with open(model_name, 'wb') as model_file:
    pickle.dump(gnb, model_file)
  print('model saved')
  name_param[model_name] = 'default params'
  return name_param

#### classifier training

In [68]:
main_dict = {}
cluster = break_cluster(train_data_pca,train_cluster_labels,train_classifier_target)
for label,feature,target in cluster:
  print(label, len(feature),len(target))
  print(np.unique(target))
  print(type(target))
  log_dict = logistic_train(label,feature,target)
  gd_dict = sgd_train(label,feature,target)
  rfc_dict = random_forest_train(label,feature,target)
  xgb_dict = xgb_train(label,feature,target)
  cnb_dict = complement_nb_train(label,feature,target)
  gnb_dict = gaussian_nb_train(label,feature,target)
  main_dict[label] = [log_dict,sgd_dict,rfc_dict,xgb_dict,cnb_dict,gnb_dict]

save_dict = {}
for key,val in main_dict.items():
  save_dict[str(key)] = val

# writing out dict as json
with open('classiier_id_dict.json', 'w') as clf_dict:
  json.dump(save_dict, clf_dict)

NameError: ignored

In [69]:
save_dict = {}
for key,val in main_dict.items():
  save_dict[str(key)] = val

# writing out dict as json
with open('classiier_id_dict.json', 'w') as clf_dict:
  json.dump(save_dict, clf_dict)

### Auto Encoder generative Model


In [73]:
def auto_feature(file_string):
  if file_string == 'train':
    feature_col = train_original_previous_day_fire.values
    target_col = train_original_next_day_fire.values
  if file_string == 'test':
    feature_col = test_original_previous_day_fire.values
    target_col = test_original_next_day_fire.values
  if file_string == 'eval':
    feature_col = eval_original_previous_day_fire.values
    target_col = eval_original_next_day_fire.values

  chunk_count = int(len(feature_col)/(64*64))
  feature_list = np.array_split(feature_col, chunk_count)
  target_list = np.array_split(target_col, chunk_count)

  for a_feature,a_target in zip(feature_list,target_list):
    yield a_feature.reshape(64,64), a_target.reshape(64,64)

#### Training Auto-Encoder

In [ ]:
%%time
gpus = tf.config.experimental.list_physical_devices('GPU')
parameters = {
    'encoding_dim': [64],  # Number of units in the bottleneck layer
    'epochs': [10],  # Number of training epochs
}

input_dim = 64 # input feature count
for params in ParameterGrid(parameters):
  encoding_dim = params['encoding_dim'] #the number of features post reduction
  input_data = Input(shape=(input_dim,))

  encoded = Dense(encoding_dim*2, activation='relu')(input_data)
  encoded = Dense(encoding_dim, activation='relu')(encoded)
  decoded = Dense(encoding_dim*2, activation='relu')(encoded)
  decoded = Dense(input_dim, activation='sigmoid')(decoded)
  autoencoder = Model(input_data, decoded)

# Compile the autoencoder
  autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Training the autoencoder on GPU in batch
  with tf.device('/GPU:0'):
    for feature,target in auto_feature('train'):
      autoencoder.fit(feature,target, epochs=params['epochs'], shuffle=True)
  file_string = 'autoencoder_generator'+'.h5'
  autoencoder.save(file_string)

In [ ]:
autoencoder = load_model('/content/autoencoder_generator.h5')
for threshold in [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]:
  accuracy_list = []
  for a_predict, a_target in auto_feature('eval'):
    binary_prediction = np.where(a_predict > threshold, 1, 0)
    accuracy = accuracy_score(a_target, binary_prediction)
    accuracy_list.append(accuracy)

mean_dict = {}
for key, val in accuracy_dict.items():
  mean_dict[key] =  np.mean(val)

### Model Evaluation

#### Function to evaluate the dataset for only when ground truth is 1 [Fire]

In [ ]:
def pos_fire_break(feature_data, label_data):
  '''
  Split the dataset based on present fire 0 and 1 for prediction/accuracy comparisons
  '''
  ind = np.where(label_data == 1)[0]
  reduced_feature_data = feature_data[ind]
  reduced_label_data = label_data[ind]
  return reduced_feature_data, reduced_label_data

#### All Supervised Model evaluations

In [ ]:
base_path = '/content'
model_dict = {}
model_list = ['ComplementNB_1','guassianNB_1','logistic_model_1','random_forest_model_1','random_forest_model_2','random_forest_model_3','SGD_model_1','SGD_model_2','SGD_model_3','xgb_model_1','xgb_model_2','xgb_model_3']


cl = break_cluster(eval_data_pca,eval_cluster_labels,eval_classifier_target)

for label,feature,target in cl:
  feats, tar = pos_fire_break(feature, target)
  for m in model_list:
    #Try to open pickled model data - some may not open due to truncation
    #This could be an issue from how we allocated cluster data and then trained the model
    try:
      with open(base_path+'/cluster_'+str(label)+"_"+m,'rb') as file:
        temp_model = pickle.load(file) #load in model
    except:
      print(temp_model, "pickle data was truncated - cannot run")
      break

    #truncated name for use in the model_dict
    truncated_file_name = str(file).split('/')[-1].split('>')[0].split('\'')[0]

    #Try to run 5 Fold CV for all eval data
    kf = KFold(n_splits=5)
    fold = 1
    acc,pre,rec,f1 = [],[],[],[]
    for eval_ind in kf.split(feature):
      y_pred = temp_model.predict(feature)
      acc.append(accuracy_score(target,y_pred))
      pre.append(precision_score(target,y_pred))
      rec.append(recall_score(target,y_pred))
      f1.append(f1_score(target,y_pred))

      fold += 1
    cv_scores = {'accuracy':np.mean(acc),'precision':np.mean(pre),'recall':np.mean(rec),'F1':np.mean(f1)}
    model_dict[truncated_file_name] = cv_scores

    #Try to run 5 fold CV on reduced data
    #Failure of run may be due to lack in present fire data labels
    try:
      kf = KFold(n_splits=5)
      fold = 1
      acc,pre,rec,f1 = [],[],[],[]
      for eval_ind in kf.split(feats):
        y_pred = temp_model.predict(feats)
        acc.append(accuracy_score(tar,y_pred))
        pre.append(precision_score(tar,y_pred))
        rec.append(recall_score(tar,y_pred))
        f1.append(f1_score(tar,y_pred))
        fold += 1
      cv_scores_2 = {'accuracy':np.mean(acc),'precision':np.mean(pre),'recall':np.mean(rec),'F1':np.mean(f1)}

      model_dict[truncated_file_name+'_reduced'] = cv_scores_2
    except:
      print("Failed Reduced CV for Cluster",label,"Model",m)

model_df = pd.DataFrame(model_dict)
#Clear up some memory
del temp_model
del cl
del feats
del tar

#### Creating Evaluation DF for visualization

In [ ]:
# Mean together accross different clusters
model_list = ['ComplementNB_1','guassianNB_1','logistic_model_1','random_forest_model_1','random_forest_model_2','random_forest_model_3','SGD_model_1','SGD_model_2','SGD_model_3','xgb_model_1','xgb_model_2','xgb_model_3']
model_grouped_dict = {}
for mod in model_list:
  mod_list = []
  mod_reduced_list = []
  for each in model_df.columns:
    if str(mod) in str(each) and "reduced" in str(each):
      mod_reduced_list.append(each)
    elif str(mod) in str(each):
      mod_list.append(each)
  model_grouped_dict[mod] = model_df[mod_list].mean(axis=1)
  model_grouped_dict[mod+'_reduced'] = model_df[mod_reduced_list].mean(axis=1)
#Finalized dataframe of important information
DF = pd.DataFrame(model_grouped_dict)

In [ ]:
reduced_cols = [col for col in DF.columns if 'reduced' in col]
non_red_cols = [col for col in DF.columns if 'reduced' not in col]
print("Reduced data metrics: \n",DF[reduced_cols].idxmax(axis=1))
print("Data metrics: \n",DF[non_red_cols].idxmax(axis=1))

#Best models for reduced data
red_scores = DF[set(DF[reduced_cols].idxmax(axis=1))]
print(red_scores)

#Best models for data
non_red_scores = DF[set(DF[non_red_cols].idxmax(axis=1))]
print(non_red_scores)

#### Supervised Model visualization

In [ ]:
fig, ax = plt.subplots(nrows = 4,ncols=1,figsize=(20,15),sharex=True)
row = -1
for metric in ['accuracy','precision','recall','F1']:
  X = (DF.columns)
  row += 1
  y = DF.iloc[row]

  ax[row].bar(X,y)
  ax[row].set_ylabel(metric)


plt.xticks(rotation=90)
plt.tight_layout()

plt.show()

### Feature Ablation

In [ ]:
scalar = '/content/standard_scalar_model'
pca_chosen = '/content/pca_model_8'
kmeans_chosen = '/content/kmean_model_1'

#standard_scalar_model
with open(scalar, 'rb') as ss_file:
    loaded_scalar_model = pickle.load(ss_file)

# pca model chosen:
with open(pca_chosen, 'rb') as pca_file:
    loaded_pca_model = pickle.load(pca_file)

with open('cluster_3_ComplementNB_1','rb') as file:
  loaded_model = pickle.load(file)


print('initializing test data scaling')
test_data_scaled


In [ ]:
ablated_accuracies = []
# Perform feature ablation
num_features = test_data_scaled.shape[1]

for feature in range(num_features):
    X_ablated = test_data_scaled.copy()
    X_ablated[:, feature] = 0  # Set the feature values to 0

    test_data_pca = loaded_pca_model.transform(X_ablated)
    ablated_accuracy = accuracy_score(test_classifier_target,loaded_model.predict(test_data_pca))
    ablated_accuracies.append(ablated_accuracy)

    print("Feature {}: Ablated Accuracy: {:.4f}".format(feature, ablated_accuracy))

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.bar(range(num_features), ablated_accuracies[:num_features])
plt.xlabel('Feature Index')
plt.ylabel('Accuracy')
plt.title('Feature Ablation')
plt.xticks(range(num_features))
plt.show()

In [ ]:
class_means = loaded_model.theta_
class_std = loaded_model.sigma_
importance_scores = np.mean(class_means / class_std, axis=0)
sorted_indicies = np.argsort(importance_scores)[::-1]
feature_names = ["Feature"+str(x) for x in range(0,19)]
print("Feature Importance:")
for i in sorted_indicies:
    print(f"{feature_names[int(i)]}: {importance_scores[int(i)]}")